# Quickstart

This notebook shows how to use PySTAC to read through the public Sentinel catalog, and grab information for a single band's file.

First, we want to hook into PySTAC to allow for reading of HTTP STAC items, as described in [the STAC_IO Concepts docs](concepts.html#using-stac-io). 

__Note:__ this requires the [requests](https://requests.kennethreitz.org/en/master) library be installed.

In [1]:
from urllib.parse import urlparse
import requests
from pystac import STAC_IO

def requests_read_method(uri):
    parsed = urlparse(uri)
    if parsed.scheme.startswith('http'):
        return requests.get(uri).text
    else:
        return STAC_IO.default_read_text_method(uri)

STAC_IO.read_text_method = requests_read_method

We can then read the STAC catalog located at the publicly available endpoint hosted by AWS:

In [3]:
from pystac import Catalog

cat = Catalog.from_file('https://sentinel-stac.s3.amazonaws.com/catalog.json')

There are a lot of items in this catalog; crawling through it all would take a significant amount of time. Here, we lean on the fact that [link resolution is lazy](concepts.html#lazy-resolution-of-stac-objects) and get to a catalog that contains items:

In [4]:
while len(cat.get_item_links()) == 0:
    print('Crawling through {}'.format(cat))
    cat = next(cat.get_children())

Crawling through <Catalog id=sentinel-stac>
Crawling through <Collection id=sentinel-2-l1c>
Crawling through <Catalog id=9>
Crawling through <Catalog id=V>


We can print some information about the catalog, including how many children it has:

In [8]:
print(cat.description)
print('Contains {} items.'.format(len(cat.get_item_links())))

XK catalog
Contains 388 items.


Let's grab the first item, check out it's cloud cover, and start exploring the assets.

In [9]:
item = next(cat.get_items())

In [10]:
item.cloud_cover

41.52

In [11]:
for asset_key in item.assets:
    asset = item.assets[asset_key]
    print('{}: {} ({})'.format(asset_key, asset.href, asset.media_type))

thumbnail: https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/13/0/preview.jpg (None)
info: https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/13/0/tileInfo.json (None)
metadata: https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/9/V/XK/2017/10/13/0/metadata.xml (None)
tki: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/TKI.jp2 (image/jp2)
B01: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B01.jp2 (image/jp2)
B02: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B02.jp2 (image/jp2)
B03: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B03.jp2 (image/jp2)
B04: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B04.jp2 (image/jp2)
B05: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B05.jp2 (image/jp2)
B06: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B06.jp2 (image/jp2)
B07: https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9

We can use the `to_dict()` method to convert an Asset, or any PySTAC object, into a dictionary:

In [12]:
asset = item.assets['B03']
asset.to_dict()

{'href': 'https://sentinel-s2-l1c.s3.amazonaws.com/tiles/9/V/XK/2017/10/13/0/B03.jp2',
 'type': 'image/jp2',
 'title': 'Band 3 (green)',
 'eo:bands': [2]}

Here the asset uses the band information associated with it's item:

In [13]:
asset.get_bands()[0].to_dict()

{'name': 'B03',
 'common_name': 'green',
 'gsd': 10.0,
 'center_wavelength': 0.56,
 'full_width_half_max': 0.045}